<h3>建立北捷路線各站關係圖，每站須記載中文站名，站名英文代碼，每站均各有隨機成本，站與站間亦各有隨機成本等資訊</h3>

In [11]:
import numpy
import pandas
from glob import glob
import jaydebeapi

In [13]:
from pyArango.connection import *
from pyArango.query import *
import pyArango.validation as VAL

In [14]:
import networkx
from networkx.algorithms.shortest_paths.weighted import single_source_dijkstra

In [15]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

In [ ]:
# my_font = FontProperties(fname=r"C:\Windows\Fonts/Simsun.ttc", size=14)
# my_font.get_name()

In [ ]:
#dir(my_font)

<h3>讀取各捷運線的資料檔

In [16]:
data_folder = 'C:/Users/user/anaconda3/Projects/data'

data_files = glob('%s/MRT_??.csv' % data_folder) #用 % 指定位址
data_files

['C:/Users/user/anaconda3/Projects/data\\MRT_ G.csv',
 'C:/Users/user/anaconda3/Projects/data\\MRT_ O.csv',
 'C:/Users/user/anaconda3/Projects/data\\MRT_ R.csv',
 'C:/Users/user/anaconda3/Projects/data\\MRT_ Y.csv',
 'C:/Users/user/anaconda3/Projects/data\\MRT_BL.csv',
 'C:/Users/user/anaconda3/Projects/data\\MRT_BR.csv']

In [17]:
data_G = pandas.read_csv(data_files[0], encoding='CP950').fillna('')
data_O = pandas.read_csv(data_files[1], encoding='CP950').fillna('')
data_R = pandas.read_csv(data_files[2], encoding='CP950').fillna('')
data_Y = pandas.read_csv(data_files[3], encoding='CP950').fillna('')
data_BL = pandas.read_csv(data_files[4], encoding='CP950').fillna('')
data_BR = pandas.read_csv(data_files[5], encoding='CP950').fillna('')
data_BR

StationID Line Station PreStation NextStation  Joint Transit
0       BR01  文湖線     動物園                   BR02      0        
1       BR02  文湖線      木柵       BR01        BR03      0        
2       BR03  文湖線    萬芳社區       BR02        BR04      0        
3       BR04  文湖線    萬芳醫院       BR03        BR05      0        
4       BR05  文湖線      辛亥       BR04        BR06      0        
5       BR06  文湖線      麟光       BR05        BR07      0        
6       BR07  文湖線     六張犁       BR06        BR08      0        
7       BR08  文湖線    科技大樓       BR07        BR09      0        
8       BR09  文湖線      大安       BR08        BR10      1     R05
9       BR10  文湖線    忠孝復興       BR09        BR11      1    BL15
10      BR11  文湖線    南京復興       BR10        BR12      1     G16
11      BR12  文湖線    中山國中       BR11        BR13      0        
12      BR13  文湖線    松山機場       BR12        BR14      0        
13      BR14  文湖線      大直       BR13        BR15      0        
14      BR15  文湖線     劍南路       BR14        BR16      0        
15      BR16  文湖線      西湖       BR15        BR17      0        
16      BR17  文湖線      港墘       BR16        BR18      0        
17      BR18  文湖線      文德       BR17        BR19      0        
18      BR19  文湖線      內湖       BR18        BR20      0        
19      BR20  文湖線    大湖公園       BR19        BR21      0        
20      BR21  文湖線      葫洲       BR20        BR22      0        
21      BR22  文湖線      東湖       BR21        BR23      0        
22      BR23  文湖線  南港軟體園區       BR22        BR24      0        
23      BR24  文湖線   南港展覽館       BR23                  1    BL23

<h3>讀取站名及代碼</h3>

In [43]:
node_info=pandas.read_csv('C:/Users/user/anaconda3/Projects/data/node_Info.csv', encoding='CP950').fillna('')
node_info

StationID Station
0        BR01     動物園
1        BR02      木柵
2        BR03    萬芳社區
3        BR04    萬芳醫院
4        BR05      辛亥
..        ...     ...
130       Y18     頭前庄
131       Y19      幸福
132       Y20  新北產業園區
133      G03A     小碧潭
134      R22A     新北投

[135 rows x 2 columns]

<h3>連接 ArangoDB

In [6]:
db_conn= Connection(
        arangoURL = 'http://localhost:8529',
        username = 'emprogria',
        password = 'emprogria')
db = db_conn['Emprogria']
db_collection = [
    db.collections['MRT_BR'],
    db.collections['MRT_R'],
    db.collections['MRT_G'],
    db.collections['MRT_O'],
    db.collections['MRT_BL'],
    db.collections['MRT_Y'],
    db.collections['node_info']
]

<h3>清空 ArangoDB 中各捷運線的 Collection

In [48]:
for i in range(len(db_collection)):
    db_result = db_collection[i].empty()

<h3>建立各捷運線的 Dictionary，並將存放 Dictionary 的 List 匯出 txt，在外部將 txt 檔轉換成 json 檔匯入 ArangoDB

<h4>文湖線 BR

In [49]:
BR = list()
for i in range(data_BR.shape[0]):
    info_BR = {
    '_key': data_BR['StationID'][i],
    'Lane': data_BR['Line'][i],
    'StationName': data_BR['Station'][i],
    'PreStation': data_BR['PreStation'][i],
    'NextStation': data_BR['NextStation'][i],
    'Joint': data_BR['Joint'][i],
    'Transit': data_BR['Transit'][i],
    'Weight': numpy.random.randint(1, 100)
    }
    BR.append(info_BR)
with open('MRT_BR.txt', 'w', encoding='utf-8') as f:
    f.write(str(BR))

<h4>淡水信義線 R

In [31]:
R = list()
for i in range(data_R.shape[0]):
    info_R = {
    '_key': data_R['StationID'][i],
    'Lane': data_R['Line'][i],
    'StationName': data_R['Station'][i],
    'PreStation': data_R['PreStation'][i],
    'NextStation': data_R['NextStation'][i],
    'Joint': data_R['Joint'][i],
    'Transit': data_R['Transit'][i],
    'Weight': numpy.random.randint(1, 100)
    }
    R.append(info_R)
with open('MRT_R.txt', 'w', encoding='utf-8') as f:
    f.write(str(R))

<h4>松山新店線 G

In [32]:
G = list()
for i in range(data_G.shape[0]):
    info_G = {
    '_key': data_G['StationID'][i],
    'Lane': data_G['Line'][i],
    'StationName': data_G['Station'][i],
    'PreStation': data_G['PreStation'][i],
    'NextStation': data_G['NextStation'][i],
    'Joint': data_G['Joint'][i],
    'Transit': data_G['Transit'][i],
    'Weight': numpy.random.randint(1, 100)
    }
    G.append(info_G)
with open('MRT_G.txt', 'w', encoding='utf-8') as f:
    f.write(str(G))

<h4>中和新蘆線 O

In [33]:
O = list()
for i in range(data_O.shape[0]):
    info_O = {
    '_key': data_O['StationID'][i],
    'Lane': data_O['Line'][i],
    'StationName': data_O['Station'][i],
    'PreStation': data_O['PreStation'][i],
    'NextStation': data_O['NextStation'][i],
    'Joint': data_O['Joint'][i],
    'Transit': data_O['Transit'][i],
    'Weight': numpy.random.randint(1, 100)
    }
    O.append(info_O)
with open('MRT_O.txt', 'w', encoding='utf-8') as f:
    f.write(str(O))

<h4>板南線 BL

In [44]:
BL = list()
for i in range(data_BL.shape[0]):
    info_BL = {
    '_key': data_BL['StationID'][i],
    'Lane': data_BL['Line'][i],
    'StationName': data_BL['Station'][i],
    'PreStation': data_BL['PreStation'][i],
    'NextStation': data_BL['NextStation'][i],
    'Joint': data_BL['Joint'][i],
    'Transit': data_BL['Transit'][i],
    'Weight': numpy.random.randint(1, 100)
    }
    BL.append(info_BL)
with open('MRT_BL.txt', 'w', encoding='utf-8') as f:
    f.write(str(BL))

<h4>環狀線 Y

In [22]:
Y = list()
for i in range(data_Y.shape[0]):
    info_Y = {
    '_key': data_Y['StationID'][i],
    'Lane': data_Y['Line'][i],
    'StationName': data_Y['Station'][i],
    'PreStation': data_Y['PreStation'][i],
    'NextStation': data_Y['NextStation'][i],
    'Joint': data_Y['Joint'][i],
    'Transit': data_Y['Transit'][i],
    'Weight': numpy.random.randint(1, 100)
    }
    Y.append(info_Y)
with open('MRT_Y.txt', 'w', encoding='utf-8') as f:
    f.write(str(Y))

In [16]:
node_info.shape[0]

135

In [18]:
#所有站名代碼
nodes = list()
for i in range(node_info.shape[0]):
    node = {
    '_key': node_info['StationID'][i],
    'StationID': node_info['StationID'][i],
    'StationName': node_info['Station'][i]
    }
    nodes.append(node)
with open('node.txt', 'w', encoding='utf-8') as f:
    f.write(str(nodes))

<h3>連結DBeaver</h3>

In [44]:
dbConnection = jaydebeapi.connect(
    "org.h2.Driver",
    "jdbc:h2:C:/Users/user/anaconda3/Projects/data/H2",
    ["SA", ""],
    "C:/Java/h2/bin/h2-1.4.200.jar")

dbCursor = dbConnection.cursor()

In [6]:
#G
_sql = [
    "CREATE TABLE IF NOT EXISTS",
    "MRT_G(StationID CHAR, Line CHAR,",
    "Station CHAR, PreStation CHAR, NextStation CHAR,",
    "Joint NUMBER, Transit CHAR);",
]
sql = ' '.join(_sql)
dbCursor.execute(sql)
_sql = [
    "INSERT INTO",
    "MRT_G(STationID, Line,",
    "Station, PreStation, NextStation,",
    "Joint, Transit)",
    "VALUES('%s','%s', '%s', '%s', '%s', '%s', '%s');"
]
sql = ' '.join(_sql)
for i in range(data_G.shape[0]):
    row = data_G.iloc[i]

    insert_sql = sql % (row['StationID'],
                        row['Line'],
                        row['Station'],
                        row['PreStation'],
                        row['NextStation'],
                        row['Joint'],
                        row['Transit']
                       )

    dbCursor.execute(insert_sql)

In [35]:
#O
_sql = [
    "CREATE TABLE IF NOT EXISTS",
    "MRT_O(StationID CHAR, Line CHAR,",
    "Station CHAR, PreStation CHAR, NextStation CHAR,",
    "Joint NUMBER, Transit CHAR);",
]
sql = ' '.join(_sql)
dbCursor.execute(sql)
_sql = [
    "INSERT INTO",
    "MRT_O(STationID, Line,",
    "Station, PreStation, NextStation,",
    "Joint, Transit)",
    "VALUES('%s','%s', '%s', '%s', '%s', '%s', '%s');"
]
sql = ' '.join(_sql)
for i in range(data_O.shape[0]):
    row = data_O.iloc[i]

    insert_sql = sql % (row['StationID'],
                        row['Line'],
                        row['Station'],
                        row['PreStation'],
                        row['NextStation'],
                        row['Joint'],
                        row['Transit']
                       )

    dbCursor.execute(insert_sql)

In [38]:
#R
_sql = [
    "CREATE TABLE IF NOT EXISTS",
    "MRT_R(StationID CHAR, Line CHAR,",
    "Station CHAR, PreStation CHAR, NextStation CHAR,",
    "Joint NUMBER, Transit CHAR);",
]
sql = ' '.join(_sql)
dbCursor.execute(sql)
_sql = [
    "INSERT INTO",
    "MRT_R(STationID, Line,",
    "Station, PreStation, NextStation,",
    "Joint, Transit)",
    "VALUES('%s','%s', '%s', '%s', '%s', '%s', '%s');"
]
sql = ' '.join(_sql)
for i in range(data_R.shape[0]):
    row = data_R.iloc[i]

    insert_sql = sql % (row['StationID'],
                        row['Line'],
                        row['Station'],
                        row['PreStation'],
                        row['NextStation'],
                        row['Joint'],
                        row['Transit']
                       )

    dbCursor.execute(insert_sql)

In [39]:
#Y
_sql = [
    "CREATE TABLE IF NOT EXISTS",
    "MRT_Y(StationID CHAR, Line CHAR,",
    "Station CHAR, PreStation CHAR, NextStation CHAR,",
    "Joint NUMBER, Transit CHAR);",
]
sql = ' '.join(_sql)
dbCursor.execute(sql)
_sql = [
    "INSERT INTO",
    "MRT_Y(STationID, Line,",
    "Station, PreStation, NextStation,",
    "Joint, Transit)",
    "VALUES('%s','%s', '%s', '%s', '%s', '%s', '%s');"
]
sql = ' '.join(_sql)
for i in range(data_Y.shape[0]):
    row = data_Y.iloc[i]

    insert_sql = sql % (row['StationID'],
                        row['Line'],
                        row['Station'],
                        row['PreStation'],
                        row['NextStation'],
                        row['Joint'],
                        row['Transit']
                       )

    dbCursor.execute(insert_sql)

In [40]:
#BL
_sql = [
    "CREATE TABLE IF NOT EXISTS",
    "MRT_BL(StationID CHAR, Line CHAR,",
    "Station CHAR, PreStation CHAR, NextStation CHAR,",
    "Joint NUMBER, Transit CHAR);",
]
sql = ' '.join(_sql)
dbCursor.execute(sql)
_sql = [
    "INSERT INTO",
    "MRT_BL(STationID, Line,",
    "Station, PreStation, NextStation,",
    "Joint, Transit)",
    "VALUES('%s','%s', '%s', '%s', '%s', '%s', '%s');"
]
sql = ' '.join(_sql)
for i in range(data_BL.shape[0]):
    row = data_BL.iloc[i]

    insert_sql = sql % (row['StationID'],
                        row['Line'],
                        row['Station'],
                        row['PreStation'],
                        row['NextStation'],
                        row['Joint'],
                        row['Transit']
                       )

    dbCursor.execute(insert_sql)

In [41]:
#BR
_sql = [
    "CREATE TABLE IF NOT EXISTS",
    "MRT_BR(StationID CHAR, Line CHAR,",
    "Station CHAR, PreStation CHAR, NextStation CHAR,",
    "Joint NUMBER, Transit CHAR);",
]
sql = ' '.join(_sql)
dbCursor.execute(sql)
_sql = [
    "INSERT INTO",
    "MRT_BR(STationID, Line,",
    "Station, PreStation, NextStation,",
    "Joint, Transit)",
    "VALUES('%s','%s', '%s', '%s', '%s', '%s', '%s');"
]
sql = ' '.join(_sql)
for i in range(data_BR.shape[0]):
    row = data_BR.iloc[i]

    insert_sql = sql % (row['StationID'],
                        row['Line'],
                        row['Station'],
                        row['PreStation'],
                        row['NextStation'],
                        row['Joint'],
                        row['Transit']
                       )

    dbCursor.execute(insert_sql)

In [52]:
#所有站名代碼
_sql = [
    "CREATE TABLE IF NOT EXISTS",
    "node_info(StationID CHAR, Station CHAR);",
]
sql = ' '.join(_sql)
dbCursor.execute(sql)
_sql = [
    "INSERT INTO",
    "node_info(StationID, Station)",
    "VALUES('%s','%s');"
]
sql = ' '.join(_sql)
for i in range(node_info.shape[0]):
    row = node_info.iloc[i]
    insert_sql = sql % (row['StationID'],
                        row['Station']
                       )

    dbCursor.execute(insert_sql)

In [53]:
dbCursor.close()
dbConnection.close()